# Lab 3: Alignment-based metrics in Machine Learning

* Author: Romain Tavenard (@rtavenar)
* License: CC-BY-NC-SA

A lab session from a course on Machine Learning for Time Series at ENSAI.
One can find lecture notes for this course [there](https://rtavenar.github.io/ml4ts_ensai/).

In [1]:
# Here, we install an unstable version of `tslearn` that integrates Keras 3
!pip install git+https://github.com/tslearn-team/tslearn.git@keras3

  Cloning https://github.com/tslearn-team/tslearn.git (to revision keras3) to /private/var/folders/zb/0wn4gjps68sdtn6rzz4s6hkr0000gq/T/pip-req-build-gprb1adi
  Running command git clone --filter=blob:none --quiet https://github.com/tslearn-team/tslearn.git /private/var/folders/zb/0wn4gjps68sdtn6rzz4s6hkr0000gq/T/pip-req-build-gprb1adi
  Running command git checkout -b keras3 --track origin/keras3
  Switched to a new branch 'keras3'
  branch 'keras3' set up to track 'origin/keras3'.
  Resolved https://github.com/tslearn-team/tslearn.git to commit aecce61991026c705ecc9412da52df0e04202452
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from tslearn.metrics import dtw, soft_dtw
from tslearn.barycenters import dtw_barycenter_averaging, softdtw_barycenter

# Data loading

**Question #1.** Using the [`CachedDatasets`](https://tslearn.readthedocs.io/en/stable/gen_modules/datasets/tslearn.datasets.CachedDatasets.html#tslearn.datasets.CachedDatasets)
utility from ``tslearn``, load the "Trace" time series dataset.
What are the dimensions of an array storing a time series dataset?
Create a new dataset `X_subset` made of 50 random time series from classes indexed 1 to 3 (`y_train < 4`)
in the training set.

In [3]:
from tslearn.datasets import CachedDatasets

data_loader = CachedDatasets()

X_train, y_train, _, _ = data_loader.load_dataset("Trace")
X_train = X_train[y_train < 4]
X_train.shape

(69, 275, 1)

In [5]:
indices = np.random.choice(X_train.shape[0], size=50)
X_subset = X_train[indices]
X_subset.shape

(50, 275, 1)

# $k$-means clustering

**Question #2.** Implement the Lloyd's algorithm for a $k$-means that would use soft-DTW as 
its base metric.
You can rely on ``tslearn`` functions (see imports above) for "distance" computations and barycenter
estimation.
Your function should return both the current assignments and the barycenters.
Check that it runs smoothly for a few iterations on `X_subset` (leave quantitative evaluation aside for now).

In [7]:
def kmeans_soft_dtw(X, gamma, k, max_iter=10):
    n_timeseries, n_timestamps, n_features = X.shape

    # Init barycenters at random
    barycenters = X[np.random.randint(n_timeseries, size=k)]

    for e in range(max_iter):
        # Assign a cluster to each time series
        assign = np.zeros((n_timeseries, ), dtype=int)
        for i in range(n_timeseries):
            distances = [soft_dtw(X[i], b, gamma=gamma) for b in barycenters]
            assign[i] = np.argmin(distances)
        # Update centroids (barycenters) for each cluster
        for j in range(k):
            barycenters[j] = softdtw_barycenter(X[assign == j], gamma=gamma)
    
    return assign, barycenters

kmeans_soft_dtw(X_subset, gamma=.1, k=3, max_iter=3)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


(array([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 1, 2, 0, 2, 1, 0, 0, 1, 1,
        1, 0, 0, 0, 0, 1, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 1,
        0, 0, 2, 0, 2, 0]),
 array([[[ 6.80078051e-01],
         [ 6.82074051e-01],
         [ 6.80222133e-01],
         [ 6.78536298e-01],
         [ 6.77719356e-01],
         [ 6.77210924e-01],
         [ 6.76674673e-01],
         [ 6.76225108e-01],
         [ 6.75807466e-01],
         [ 6.75694886e-01],
         [ 6.76014750e-01],
         [ 6.76637461e-01],
         [ 6.77365452e-01],
         [ 6.78151300e-01],
         [ 6.78804395e-01],
         [ 6.79189046e-01],
         [ 6.79368546e-01],
         [ 6.79301563e-01],
         [ 6.79058998e-01],
         [ 6.78557117e-01],
         [ 6.78015731e-01],
         [ 6.77458614e-01],
         [ 6.76922968e-01],
         [ 6.76400382e-01],
         [ 6.76076892e-01],
         [ 6.75934388e-01],
         [ 6.76111016e-01],
         [ 6.76657401e-01],
         [ 6.77525980e-01],
         [ 6

**Question #3.** Implement the Lloyd's algorithm for a $k$-means that would use **DTW** as 
its base metric.
You can rely on ``tslearn`` functions (see imports above) for "distance" computations and barycenter
estimation.
Your function should return both the current assignments and the barycenters.
Check that it runs smoothly for a few iteration on `X_subset` (leave quantitative evaluation aside for now).

In [8]:
def kmeans_dtw(X, k, max_iter=10):
    n_timeseries, n_timestamps, n_features = X.shape

    # Init barycenters at random
    barycenters = X[np.random.randint(n_timeseries, size=k)]

    for e in range(max_iter):
        # Assign a cluster to each time series
        assign = np.zeros((n_timeseries, ), dtype=int)
        for i in range(n_timeseries):
            distances = [dtw(X[i], b) for b in barycenters]
            assign[i] = np.argmin(distances)
        # Update centroids (barycenters) for each cluster
        for j in range(k):
            barycenters[j] = dtw_barycenter_averaging(X[assign == j])
    
    return assign, barycenters

kmeans_dtw(X_subset, k=3, max_iter=3)

(array([0, 1, 0, 1, 0, 0, 1, 2, 0, 0, 2, 2, 1, 2, 2, 0, 2, 2, 1, 0, 2, 2,
        2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 0, 1, 2, 1, 1, 0, 0, 0, 0, 2, 0, 2,
        0, 0, 2, 1, 2, 1]),
 array([[[ 6.13360769e-01],
         [ 6.38503846e-01],
         [ 6.75222708e-01],
         [ 6.12869000e-01],
         [ 5.88681111e-01],
         [ 6.07154211e-01],
         [ 6.25197692e-01],
         [ 5.56924348e-01],
         [ 6.15803750e-01],
         [ 6.59332500e-01],
         [ 6.03639583e-01],
         [ 6.22298182e-01],
         [ 6.34867857e-01],
         [ 6.04735909e-01],
         [ 6.35083529e-01],
         [ 6.26901250e-01],
         [ 6.04219474e-01],
         [ 6.30862632e-01],
         [ 5.71920400e-01],
         [ 6.20492609e-01],
         [ 6.38282105e-01],
         [ 5.82501562e-01],
         [ 6.22830800e-01],
         [ 5.61354571e-01],
         [ 6.17655000e-01],
         [ 6.22326250e-01],
         [ 6.25465882e-01],
         [ 6.12027222e-01],
         [ 6.27556667e-01],
         [ 6

**Question #4.** Implement a function that would assess the quality of a clustering in terms of 
intra-cluster inertia, computed using **DTW** as the base metric.
Your function should take a time series dataset, corresponding assignments and barycenters as inputs.

In [ ]:
def dtw_cost(X, assign, barycenters):
    n_timeseries, n_timestamps, n_features = X.shape

    total_cost = 0.
    # TODO
    
    return total_cost


**Question #5.** Compare your $k$-means implementations in terms of DTW inertia.
For a fair comparison, make sure that they are initialized similarly, by appropriately setting your
random number generator seeds.
What do you observe? Is that expected / Do you have an explanation for that?

# Multi-step ahead forecasting

In this section, your goal will be to implement a single-hidden-layer perceptron for time series forecasting.
Your network will be trained to minimize normalized soft-DTW[^1].

To do so, we will rely on a (very nice!) `torch`-compatible implementation of soft-DTW available in `tslearn`.

[^1]: Normalized soft-DTW (also coined soft-DTW divergence) between time series $\mathbf{x}$ and 
$\mathbf{x}^\prime$ is defined as: 
$$\text{soft-DTW}(\mathbf{x}, \mathbf{x}^\prime) - \frac{1}{2} \left( \text{soft-DTW}(\mathbf{x}, \mathbf{x}) + \text{soft-DTW}(\mathbf{x}^\prime, \mathbf{x}^\prime) \right)$$


**Question #6.** Define an MLP model that would allow training
a single-hidden-layer model using normalized soft-DTW as a criterion to be optimized.
Train your network for 200 epochs on a forecasting task that would consist, given the first 150 elements
of a time series, in predicting the next 125 ones.

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Reshape

from tslearn.metrics import SoftDTWLossPyTorch

model = Sequential([
    InputLayer(shape=(150, 1)),
    Reshape(target_shape=(150, )),

    # TODO: add your layers here...
    
    Reshape(target_shape=(125, 1))
])

**Question #7.** Use the following code block to qualitatively assess quality of your 
network's predictions.
Do not hesitate to change the time series index to visualize the result for different time series.

In [ ]:
import matplotlib.pyplot as plt

ts_index = 50

y_pred = model(X_test[:, :150, 0]).detach().numpy()

plt.figure()
plt.plot(X_test[ts_index].ravel())
plt.plot(np.arange(150, 275), y_pred[ts_index], 'r-')
plt.show()

# Shapelet models

Shapelet models are time series classifiers that rely on the presence/absence of local shapes in 
time series to make predictions.
These models will be presented in more details later in the course, since they can be seen as a variant of 
shallow convolutional models for time series.

In the following, you will train such models (using their 
[`tslearn` implementation](https://tslearn.readthedocs.io/en/stable/gen_modules/shapelets/tslearn.shapelets.LearningShapelets.html#tslearn.shapelets.LearningShapelets)) 
and visualize their learned decision boundaries.
For the sake of visualization, models made of only 2 local shapes will be learned, and the following 
function will be used to visualize them:

In [ ]:
from matplotlib import cm

def visualize_shapelet_model_2_shapelets(model, X, y):
    distances = model.transform(X)

    plt.figure()
    viridis = cm.get_cmap('viridis', 4)
    # Create a scatter plot of the 2D distances for the time series of each class.
    for i, yi in enumerate(np.unique(y)):
        plt.scatter(distances[y == yi][:, 0],
                    distances[y == yi][:, 1],
                    color=viridis(i / 3),
                    edgecolors='k',
                    label='Class {}'.format(yi))

    # Create a meshgrid of the decision boundaries
    xmin = np.min(distances[:, 0]) - 0.1
    xmax = np.max(distances[:, 0]) + 0.1
    ymin = np.min(distances[:, 1]) - 0.1
    ymax = np.max(distances[:, 1]) + 0.1
    xx, yy = np.meshgrid(np.arange(xmin, xmax, (xmax - xmin) / 200), 
                         np.arange(ymin, ymax, (ymax - ymin) / 200))
    
    weights, biases = model.get_weights('classification')
    Z = []
    for x, y in np.c_[xx.ravel(), yy.ravel()]:
        Z.append(np.argmax([biases[i] + weights[0][i]*x + weights[1][i]*y
                            for i in range(4)]))
    Z = np.array(Z).reshape(xx.shape)
    plt.contourf(xx, yy, Z / 3, cmap=viridis, alpha=0.25)

    plt.legend()
    plt.xlabel('$d(\mathbf{x}, \mathbf{s}_1)$')
    plt.ylabel('$d(\mathbf{x}, \mathbf{s}_2)$')
    plt.xlim((xmin, xmax))
    plt.ylim((ymin, ymax))
    plt.title('Distance transformed time series')
    plt.show()

Let us start with a first attempt.
The following code snippet defines a model made of 2 shapelets (=local shapes to be learnt) and train
them (as well as the corresponding linear decision boundaries) for 500 epochs:

In [ ]:
from tslearn.shapelets import LearningShapelets

shp_clf = LearningShapelets(
    n_shapelets_per_size={20: 2},  # 2 shapelets of length 20
    max_iter=500,
    verbose=0,                     # Do not print information on the evolution of the loss
    scale=False,
    random_state=42
)
shp_clf.fit(X_train, y_train)

**Question #8.** Visualize the obtained decision boundaries. 
Would you expect this model to be a decent classifier?

**Question #9.** Shapelet models usually benefit from scaling of the input time series 
(yet this is, of course, very dataset-specific).
See if this helps in our case.

**Question #10.** By default, the `tslearn` implementation uses SGD as an optimizer.
See if using `"adam"` helps in our case.

**Question #11.** Set the learning rate of the Adam optimizer to 1e-2 
(you will need to import the `Adam` class from `keras` as follows) and see if this improves the learning.

In [ ]:
from keras.optimizers import Adam

